In [1]:
import cv2
import numpy as np
cap=cv2.VideoCapture(0)

In [2]:
def draw_rect(frame):
    rows, cols, _ = frame.shape
    global total_rectangle, hand_rect_one_x, hand_rect_one_y, hand_rect_two_x, hand_rect_two_y
    
    total_rectangle=9

    hand_rect_one_x = np.array(
        [6 * rows / 20, 6 * rows / 20, 6 * rows / 20, 9 * rows / 20, 9 * rows / 20, 9 * rows / 20, 12 * rows / 20,
         12 * rows / 20, 12 * rows / 20], dtype=np.uint32)

    hand_rect_one_y = np.array(
        [9 * cols / 20, 10 * cols / 20, 11 * cols / 20, 9 * cols / 20, 10 * cols / 20, 11 * cols / 20, 9 * cols / 20,
         10 * cols / 20, 11 * cols / 20], dtype=np.uint32)

    hand_rect_two_x = hand_rect_one_x + 10
    hand_rect_two_y = hand_rect_one_y + 10

    for i in range(total_rectangle):
        cv2.rectangle(frame, (hand_rect_one_y[i], hand_rect_one_x[i]),
                      (hand_rect_two_y[i], hand_rect_two_x[i]),
                      (0, 255, 0), 1)

    return frame

In [3]:
def hand_histogram(frame):
    global hand_rect_one_x, hand_rect_two_y
    hsv_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    roi=np.zeros([90,10,3], dtype=hsv_frame.dtype)
    
    for i in range(total_rectangle):
        roi[i*10:i*10+10,0:10]=hsv_frame[hand_rect_one_x[i]:hand_rect_one_x[i] + 10,
                                          hand_rect_one_y[i]:hand_rect_one_y[i] + 10]
    hand_hist = cv2.calcHist([roi], [0, 1], None, [180, 256], [0, 180, 0, 256])
    return cv2.normalize(hand_hist, hand_hist, 0, 255, cv2.NORM_MINMAX)

In [4]:
def hist_masking(frame, hist):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    dst = cv2.calcBackProject([hsv], [0, 1], hist, [0, 180, 0, 256], 1)

    disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (31, 31))
    cv2.filter2D(dst, -1, disc, dst)

    ret, thresh = cv2.threshold(dst, 150, 255, cv2.THRESH_BINARY)

    thresh = cv2.merge((thresh, thresh, thresh))

    return cv2.bitwise_and(frame, thresh)

In [5]:
traverse_point = []
def manage_img_opr(frame,hand_hist):
    masked_frame=hist_masking(frame,hand_hist)
    imgray = cv2.cvtColor(masked_frame, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(imgray, 127, 255, 0)
    im2, contour_list, hierarchy =cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    contour_list=sorted(contour_list, key=cv2.contourArea, reverse=True)
    max_cont=contour_list[0]
    cnt_centroid = centroid(max_cont)
    cv2.circle(frame, cnt_centroid, 5, [255, 0, 255], -1)
    
    if max_cont is not None:
        hull=cv2.convexHull(max_cont,returnPoints=False)
        defects = cv2.convexityDefects(max_cont, hull)
        far_point = farthest_point(defects, max_cont, cnt_centroid)
        print("Centroid : " + str(cnt_centroid) + ", farthest Point : " + str(far_point))
        cv2.circle(frame, far_point, 5, [0, 0, 255], -1)
        if len(traverse_point) < 20:
            traverse_point.append(far_point)
        else:
            traverse_point.pop(0)
            traverse_point.append(far_point)

        draw_circles(frame, traverse_point)
        return frame
    

In [6]:
def centroid(max_contour):
    moment = cv2.moments(max_contour)
    if moment['m00'] != 0:
        cx = int(moment['m10'] / moment['m00'])
        cy = int(moment['m01'] / moment['m00'])
        return cx, cy
    else:
        return None


def farthest_point(defects, contour, centroid):
    if defects is not None and centroid is not None:
        s = defects[:, 0][:, 0]
        cx, cy = centroid

        x = np.array(contour[s][:, 0][:, 0], dtype=np.float)
        y = np.array(contour[s][:, 0][:, 1], dtype=np.float)

        xp = cv2.pow(cv2.subtract(x, cx), 2)
        yp = cv2.pow(cv2.subtract(y, cy), 2)
        dist = cv2.sqrt(cv2.add(xp, yp))

        dist_max_i = np.argmax(dist)

        if dist_max_i < len(s):
            farthest_defect = s[dist_max_i]
            farthest_point = tuple(contour[farthest_defect][0])
            return farthest_point
        else:
            return None
        
def draw_circles(frame, traverse_point):
    if traverse_point is not None:
        for i in range(len(traverse_point)):
            cv2.circle(frame, traverse_point[i], int(5 - (5 * i * 3) / 100), [0, 255, 255], -1)

In [7]:
flag=True
while True:
    ret,frame=cap.read()
    if flag:
        cv2.imshow('Hand Detector',draw_rect(frame))
        if 'c' == chr(cv2.waitKey(1) & 255):
            hand_hist=hand_histogram(frame)
            flag=False
            cv2.imshow('Hand Detector',frame)
    else:
        cv2.imshow('Hand Detector',manage_img_opr(frame,hand_hist))
        if cv2.waitKey(1)==13:
            break
    
#     c = cv2.waitKey(0)
    
cap.release()
cv2.destroyAllWindows()

Centroid : (322, 308), farthest Point : (285, 96)
Centroid : (320, 309), farthest Point : (264, 95)
Centroid : (322, 308), farthest Point : (262, 103)
Centroid : (321, 308), farthest Point : (263, 98)
Centroid : (321, 306), farthest Point : (266, 87)
Centroid : (319, 307), farthest Point : (263, 97)
Centroid : (321, 309), farthest Point : (265, 94)
Centroid : (319, 308), farthest Point : (264, 99)
Centroid : (320, 310), farthest Point : (283, 96)
Centroid : (320, 309), farthest Point : (268, 96)
Centroid : (318, 308), farthest Point : (260, 95)
Centroid : (319, 309), farthest Point : (257, 98)
Centroid : (320, 310), farthest Point : (260, 96)
Centroid : (320, 310), farthest Point : (260, 96)
Centroid : (319, 308), farthest Point : (258, 95)
Centroid : (319, 308), farthest Point : (261, 96)
Centroid : (320, 309), farthest Point : (259, 95)
Centroid : (321, 309), farthest Point : (262, 105)
Centroid : (321, 310), farthest Point : (264, 94)
Centroid : (321, 307), farthest Point : (271, 92